### Losset DYN - calculate DR indicator (dynamical part only)

In [1]:
import os
import numpy as np
from netCDF4 import Dataset
import xarray as xr

In [13]:
data_dir = '/gws/nopw/j04/kscale/USERS/dship/ERA5/'

ds_u = xr.open_dataset(os.path.join(data_dir, 'era5_u_component_of_wind_2005_12hourly.nc'))
lon = ds_u.variables['longitude'][:]
lat = ds_u.variables['latitude'][:]
lev = ds_u.variables['level'][:]
time = ds_u.variables['time'][:]
print('lon lat lev time loaded')

lon lat lev time loaded


In [3]:
# Read u, v, and omega data
u = ds_u['u']
print('u loaded')

u loaded


In [4]:
ds_v = xr.open_dataset(os.path.join(data_dir, 'era5_v_component_of_wind_2005_12hourly.nc'))
v = ds_v['v']
print('v loaded')

v loaded


In [5]:
ds_omega = xr.open_dataset(os.path.join(data_dir, 'era5_vertical_velocity_2005_12hourly.nc'))
omega = ds_omega['w']  # ERA5 'w' is in Pa s**-1 !!!
print('omega loaded')
omega = omega.chunk({"time":2})
omega

omega loaded


<xarray.DataArray 'w' (time: 62, level: 12, latitude: 721, longitude: 1440)>
dask.array<xarray-<this-array>, shape=(62, 12, 721, 1440), dtype=float32, chunksize=(2, 12, 721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 2005-01-01T05:00:00 ... 2005-01-31T17:00:00
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * level      (level) int32 100 150 200 250 300 400 500 600 700 850 925 1000
Attributes:
    standard_name:  lagrangian_tendency_of_air_pressure
    long_name:      Vertical velocity
    units:          Pa s**-1

In [6]:
# Convert omega to w
w = omega * -9.81 * 0.5  # rho = 0.5 for now, rho to be loaded in properly from file or calculated from temp and pressure
w.attrs["units"] = "m s**-1"
w

<xarray.DataArray 'w' (time: 62, level: 12, latitude: 721, longitude: 1440)>
dask.array<mul, shape=(62, 12, 721, 1440), dtype=float32, chunksize=(2, 12, 721, 1440), chunktype=numpy.ndarray>
Coordinates:
  * time       (time) datetime64[ns] 2005-01-01T05:00:00 ... 2005-01-31T17:00:00
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
  * level      (level) int32 100 150 200 250 300 400 500 600 700 850 925 1000
Attributes:
    units:    m s**-1

Select length scale $\ell_{max}$

In [7]:
Nlmax = 10

In [16]:
# Dimensions
nt = len(time)
nz = len(lev)
ny = len(lat)
nx = len(lon)

In [17]:
# Horizontal (dR) and vertical (dZ) grid step in m.
dR = abs((lon[0] - lon[1]) * 110000)
dZ = 400  # Suggests interp to 400m grid spacing required

# Horizontal size of the domain
lbox = abs((lon[-1] - lon[0]) * 110000)
dR

<xarray.Variable ()>
array(27500.)

Run $\verb|CalcPartitionIncrement.py|$

In [26]:
from CalcPartitionIncrement import CalcPartitionIncrement
CalcPartitionIncrement(dR,Nlmax)

ValueError: applied function returned data with unexpected number of dimensions. Received 1 dimension(s) but expected 0 dimensions with names: ()

In [13]:
# Calculate the partition of increments
# Calculate the smoothing function one time for all
# Normalisation

# --------- Define Global Variables ----------- #
dims = 3  # dimension of integration
face = 1.27
normphieps = 1

# Calculate the length of the increments one time for all
# Define average angle scale

dxs = dR

llcible = 1 * dxs  # convert to integers??

# Calculate the increment lengths one time for all

ibase = 0
llxt = []
llyt = []
llzt = []

lur = np.sqrt((np.array(llxt)*dR)**2 + (np.array(llyt)*dR)**2 + (np.array(llzt)*dZ)**2)
llcible = -0 * dlbd[0]

for ic in range(Nlmax):
    dll = dld[ic]
    llcible = ld[ic]
    llmov[ic] = llcible
    dllmov = dinc
    nmov = np.where((lur <= llcible+dll) & (lur > llcible-dll))[0]
    nphiinc[ic] = len(nmov)
    
    for im in range(len(nmov)):
        llx[ic, im] = llxt[nmov[im]]
        lly[ic, im] = llyt[nmov[im]]
        llz[ic, im] = llzt[nmov[im]]

# Useful for the rest
ntm = np.max(nphiinc)

# Calculate the smoothing function

deps = dxs * 1
epsl = np.arange(dxs, Nlmax*dxs+deps, deps)
Nls = len(epsl)

llmov = ld
dllmov = dR

lsingd = epsl

# Calculate phismooth and psieps
philsmooth = np.zeros((Nlmax, Nls))
phils = np.zeros((Nlmax, Nls))

for iic in range(Nlmax):
    for ic in range(Nls):
        epsr = epsl[ic]
        llt = ld[iic]
        
        dpsieps = 2 * llt * np.exp(-1/(1-(llt**2/face/epsr**2))) / normphieps / (epsr**dims) / (face*epsr**2) / (1-(llt**2/face/epsr**2))**2
        
        psieps = np.exp(-1/(1-(llt**2/face/epsr**2))) / normphieps / (epsr**dims)
        
        if llt >= np.sqrt(face) * epsr:
            dpsieps = 0
            psieps = 0
        
        philsmooth[iic, ic] = dpsieps * llt**(dims-1) * dld[iic]
        phils[iic, ic] = psieps * dld[iic]



NameError: name 'dR' is not defined

Run $\verb|CalcDRDir2D.py|$

In [11]:
# Load the field and pad it with symmetric conditions.
n1, n2, n3, nt = u.shape

# Pad Vy and Vz symmetrically everywhere
u_init = np.pad(u, ((Nlmax, Nlmax), (Nlmax, Nlmax), (0, 0), (0, 0)), mode='reflect')
v_init = np.pad(v, ((Nlmax, Nlmax), (Nlmax, Nlmax), (0, 0), (0, 0)), mode='reflect')
w_init = np.pad(w, ((Nlmax, Nlmax), (Nlmax, Nlmax), (0, 0), (0, 0)), mode='reflect')

DeltaUcubemoy = np.full((Nlmax, nt), np.nan)

# 1. Calculate the increments on these base vectors
# Check on an example; works if R=R[:,1] (column) and Z=Z[1,:] (row)

for ic in range(Nlmax):
    
    duDRt = np.full((n1, n2, n3, ntm, nt), np.nan, dtype=np.float32)
    
    for im in range(nphiinc[ic]):
        
        nlx = llx[ic, im]
        nly = lly[ic, im]
        
        # circshift(A,K) circularly shifts elements in array A by K
        # positions. If K is a vector, each element indicates the shift
        # amount in the corresponding dimension of A. np.roll should do the same thing in python
        du_l = np.roll(u_init, shift=[-nlx, -nly, 0, 0]) - u_init
        dv_l = np.roll(v_init, shift=[-nlx, -nly, 0, 0]) - v_init
        dw_l = np.roll(w_init, shift=[-nlx, -nly, 0, 0]) - w_init
        
        dusquare = du_l**2 + dv_l**2 + dw_l**2
        
        # Below is calculating component of du_l_3D along radial vector
        du_l_3D = (du_l * nlx * dR + dv_l * nly * dR + dw_l) / np.sqrt((nlx * dR)**2 + (nly * dR)**2)
        
        duDRt[:n1, :n2, :n3, im, :nt] = du_l_3D[Nlmax:Nlmax+n1, Nlmax:Nlmax+n2, :n3, :nt] * dusquare[Nlmax:Nlmax+n1, Nlmax:Nlmax+n2, :n3, :nt]
    
    # Calculate the angular average
    duDRt = np.nanmean(duDRt, axis=3)
    print('Average done')
    SulocDR[:, :, :, ic, :] = duDRt
    
    # Computation of the average DR
    DeltaUcubemoy[ic, :] = np.mean(duDRt, axis=(0, 1, 2))

# 1) Calcul de Duchon Robert [DYN] (calculation of DR DYN)

n1, n2, n3, n4, nt = SulocDR.shape

spsol = np.reshape(SulocDR / 4, (-1, n4))
spsol[np.isnan(spsol)] = 0

# DRdir2dt = spsol * philsmooth
# DRdir = np.reshape(DRdir2dt, (n1, n2, n3, Nls, nt))
# lDRdir = lsingd

NameError: name 'CalcDRDir_2D' is not defined